In [1]:
import numpy as np
import time
import os

import matplotlib.pyplot as plt
import pandas as pd
import random

import json

import keras
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras import models

os.environ["KERAS_BACKEND"] = "tensorflow"
os.environ["TF_ENABLE_GPU_GARBAGE_COLLECTION"] = 'false'
os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], False)

DEBUG = False

2022-01-29 17:24:19.890953: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-29 17:24:19.923376: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-29 17:24:19.923571: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [7]:
column_names = ['timestamp','open','close','high','low','volume','adosc','atr','macd','macd_signal','macd_hist','mfi','upper_band','middle_band','lower_band','rsi','difference_low_high','difference_open_close','target']
file = '/home/joren/Coding/cryptodata/Normalized_labelled/AAVEUSDT.bin'
modelfile = 'models/model_tiny_3'
frame_size = 240

In [8]:
# load data
field_info = [
    { "type": np.uint64, "count": 1 },
    { "type": np.double, "count": 17 },
    { "type": np.int64, "count": 1 }
]
BYTES_EIGHT = 8

def read_bin_full_file(file):
    f = open(file, 'rb')
    b = f.read(-1)

    BYTES_TO_READ = 0
    for field in field_info:
        BYTES_TO_READ += BYTES_EIGHT * field["count"]

    data = []
    BYTES_READ = 0
    for i in range(0, int(os.path.getsize(file) / BYTES_TO_READ)):
        row = []

        for idx, field in enumerate(field_info):
            row += np.frombuffer(b, dtype=field["type"], count=field["count"], offset=BYTES_READ).tolist()

            BYTES_READ += BYTES_EIGHT * field["count"]

        data.append(row)
    return np.array(data, dtype=np.float16)

In [9]:
# load model
model = models.load_model(modelfile)

2022-01-29 17:24:31.620761: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-29 17:24:31.621257: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-29 17:24:31.621483: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-29 17:24:31.621628: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [10]:
# train-test split
def df_split(df):  
    X = df.drop(columns=['timestamp','target'], axis=0).to_numpy()
    Y = df['target'].to_numpy()

    X_test = []
    y_test = []
    for i in range(frame_size, X.shape[0]): 
        X_test.append(X[i-frame_size:i])
        y_test.append(Y[i])
    X_test, y_test = np.array(X_test, dtype=np.float16), np.array(y_test, dtype=np.float16)

    return X_test, y_test

In [11]:
## prepare data for prediction
data = read_bin_full_file(file)
df2 = pd.DataFrame(data, columns=column_names)
df2.fillna(0, inplace=True)
startloc = round(len(df2)*0.75)
df = df2.iloc[startloc:]
del(df2)

In [12]:
X_test, y_test = df_split(df)

In [13]:
## predict
y_pred = np.array(model.predict(X_test), dtype=np.float16)

y_pred = np.argmax(y_pred, axis=-1, keepdims=True)
y_pred = y_pred.flatten()

print(f"""
Class. report:
{classification_report(y_test, y_pred)}
""")
cf = confusion_matrix(y_test, y_pred)
print(cf)
print(accuracy_score(y_test, y_pred) * 100)

2022-01-29 17:24:40.639171: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2037715200 exceeds 10% of free system memory.
2022-01-29 17:24:41.734557: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2037715200 exceeds 10% of free system memory.
2022-01-29 17:24:43.758114: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8300



Class. report:
              precision    recall  f1-score   support

         0.0       1.00      0.56      0.72    124121
         1.0       0.01      0.91      0.02       370
         2.0       0.01      0.88      0.02       369

    accuracy                           0.56    124860
   macro avg       0.34      0.78      0.26    124860
weighted avg       0.99      0.56      0.71    124860


[[69613 27208 27300]
 [   35   335     0]
 [   45     0   324]]
56.28063431042768


In [14]:
df = df.iloc[frame_size:]
df['predictions'] = np.array(y_pred)

In [15]:
print(df.head())

        timestamp      open     close      high       low    volume     adosc  \
375540        inf -0.002361  0.000229  0.000876 -0.001793  4.398438 -0.018082   
375541        inf -0.000038  0.000457 -0.000609 -0.000419 -0.301025  0.001405   
375542        inf  0.000343  0.001373  0.000495  0.000954 -0.122681  0.026169   
375543        inf  0.001335 -0.000839 -0.000381 -0.000343 -0.270752  0.141235   
375544        inf -0.001029 -0.001826 -0.000494 -0.000724  0.775391  0.149292   

             atr      macd  macd_signal  macd_hist       mfi  upper_band  \
375540 -0.000235 -0.169189    -0.580566   0.007973  0.445068    0.000397   
375541 -0.007648 -0.242554    -0.403809  -0.142700  0.444092    0.000345   
375542  0.012115 -0.135742    -0.288330  -0.003468  0.459473    0.000352   
375543  0.001095  0.029999    -0.179077   0.321777  0.409668    0.000227   
375544 -0.007912  0.090332    -0.102966   0.559570  0.414795    0.000412   

        middle_band  lower_band       rsi  difference_lo

In [16]:
# calculate profit/loss
total_percentage_raw = 0
percentage = 0
position = 0

for index, row in df.iterrows():

    if position == 1:
        percentage += row['close']

    if row['predictions'] == 1:
        position = 1
    elif row['predictions'] == 2:
        position = 0
        # if percentage > 0:
        #     print(f'profit: {percentage}')
        # elif percentage < 0:
        #     print(f'loss: {percentage}')
        total_percentage_raw += percentage
        percentage = 0

print(total_percentage_raw)
    

2.140297293663025


In [17]:
# calculate profit/loss with skip
total_percentage_skip = 0
percentage = 0
position = 0
skip_buy = 0
skip_sell = 0
skipcounter = 2

for index, row in df.iterrows():

    if position == 1:
        percentage += row['close']

    if row['predictions'] == 1:
        if skip_buy == skipcounter:
            position = 1
            skip_buy = 0
        skip_buy += 1
    elif row['predictions'] == 2:
        if skip_sell == skipcounter:
            skip_sell = 0
            position = 0
            # if percentage > 0:
            #     print(f'profit: {percentage}')
            # elif percentage < 0:
            #     print(f'loss: {percentage}')
            total_percentage_skip += percentage
            percentage = 0
        skip_sell += 1

print(total_percentage_skip)

0.45717912912368774


In [18]:
# calculate profit/loss
total_percentage_profit_only = 0
percentage = 0
position = 0

for index, row in df.iterrows():

    if position == 1:
        percentage += row['close']

    if row['predictions'] == 1:
        position = 1
    elif row['predictions'] == 2:
        if percentage > 0.01:
            # print(f'profit: {percentage}')
            total_percentage_profit_only += percentage
            percentage = 0
            position = 0
        

print(total_percentage_profit_only)
    

0.23173236846923828


In [19]:
# save output results to json

with open('test_results/model_tests.json', 'r') as openfile:
    # Reading from json file
    json_object = json.load(openfile)

dictionary ={
    "file" : file,
    "model" : modelfile,
    "skipcounter": skipcounter,
    "total_percentage_raw" : total_percentage_raw,
    "total_percentage_skip" : total_percentage_skip,
    "total_percentage_profit_only": total_percentage_profit_only
}

# new_object = json.dumps(dictionary, indent = 4)
json_object['results'].append(dictionary)
json_object = json.dumps(json_object, indent = 4)
with open("test_results/model_tests.json", "w") as outfile:
    outfile.write(json_object)